# Visualize Data 
**In this notebook we render a street network on a map using the Optimized Map for Analytics (OMA). We read the corresponding partitions from the OMA topology_geometry layer within a given bounding box, pass the data to the inspector which renders the geo-polylines on a map. In addition for the same tiles we also render some sample GeoJSON data showing Berlin districts.**

### Dependencies
* Catalogs: [HERE Optimized Map For Analytics](https://platform.here.com/data/hrn:here:data::olp-here:oma-3), [HERE GeoJSON Samples](https://platform.here.com/data/hrn:here:data::olp-here:here-geojson-samples)
* Layers: [topology_geometry_segment](https://platform.here.com/data/hrn:here:data::olp-here:oma-3/topology_geometry_segment), [berlin-districts](https://platform.here.com/data/hrn:here:data::olp-here:here-geojson-samples/berlin-districts)
* Languages: Python

### Workflow
* Import Packages
* Retrieve the two catalogs and layers
* Define a bounding box
* Get partitions covered by the bounding box as dataframes
* Show dataframes on a map using the inspector

## Import packages

In [1]:
import json

from shapely.geometry import shape
import geopandas as gpd

from here.platform import Platform
from here.geopandas_adapter import GeoPandasAdapter
from here.geotiles.heretile import in_bounding_box
from here.inspector.ipyleaflet import IpyleafletInspector

## Retrieve catalogs and layers

In [2]:
hp = Platform(adapter=GeoPandasAdapter())
oma_cat = hp.get_catalog('hrn:here:data::olp-here:oma-3')
oma_layer = oma_cat.get_layer('topology_geometry_segment')
samples_cat = hp.get_catalog('hrn:here:data::olp-here:here-geojson-samples')
samples_layer = samples_cat.get_layer('berlin-districts')
zoom_level = 12

In [3]:
samples_cat.open_in_portal()

## Define a bounding box

In [4]:
west = 13.35
south = 52.52
east = 13.37
north = 52.53

## Get partitions covered by the bounding box

In [5]:
partition_ids = list(in_bounding_box(west=west, south=south, east=east, north=north, level=zoom_level))

In [6]:
samples_gdf = samples_layer.read_partitions(partition_ids)
samples_gdf.head()

,partition_id,Bez,BezName,geometry
0,23618402,01,Mitte,"MULTIPOLYGON (((13.42940 52.50857, 13.42903 52..."
1,23618402,02,Friedrichshain-Kreuzberg,"MULTIPOLYGON (((13.49145 52.48827, 13.49071 52..."


In [7]:
oma_df = oma_layer.read_partitions(partition_ids)
geometry = oma_df['geometry'].apply(lambda obj: shape(json.loads(obj)))
oma_gdf = gpd.GeoDataFrame(oma_df, geometry=geometry)
oma_gdf.head()

,partition_id,identifier,start_node_partition_name,start_node_identifier,end_node_partition_name,end_node_identifier,geometry,length,partition_name
0,23618359,here:cm:segment:100000830,23618359,here:cm:node:60971159,23618359,here:cm:node:952797195,"LINESTRING Z (13.31280 52.49147 0.00000, 13.31...",175.63838,23618359
1,23618359,here:cm:segment:100000831,23618359,here:cm:node:952797195,23618359,here:cm:node:60970387,"LINESTRING Z (13.31116 52.49269 0.00000, 13.31...",60.89581,23618359
2,23618359,here:cm:segment:100000832,23618359,here:cm:node:816987233,23618359,here:cm:node:952797197,"LINESTRING Z (13.31230 52.49252 0.00000, 13.31...",72.71313,23618359
3,23618359,here:cm:segment:100000833,23618359,here:cm:node:952797197,23618359,here:cm:node:857683726,"LINESTRING Z (13.31138 52.49280 0.00000, 13.31...",47.83709,23618359
4,23618359,here:cm:segment:100000834,23618359,here:cm:node:952797195,23618359,here:cm:node:952797197,"LINESTRING Z (13.31116 52.49269 0.00000, 13.31...",19.31613,23618359


## Visualize dataframes using inspector

In [8]:
insp = IpyleafletInspector().set_zoom(zoom_level)
insp.add_features(features=oma_gdf, name="Streets", style={"color": "#5D8AA8", "weight": 1})
insp.add_features(features=samples_gdf, name="Districts", style={"color": "#AB274F"})
insp.show()

Map(center=[52.518047605, 13.37907652], controls=(LayersControl(options=['position']), ZoomControl(options=['p…

<span style="float:left; margin-top:15px;"><img src="images/HERE_Logo.png" alt="HERE Logo" height="60" width="60"></span><span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2021 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential information which may not be disclosed to others without the prior written consent of HERE.</sub></span>